# Upload to flickr

In [9]:
import os.path
import re
import sys
import tarfile
import time
import multiprocessing as mp
import itertools
from collections import Counter

import tensorflow.python.platform
from six.moves import urllib
import numpy as np
import tensorflow as tf
import h5py
import glob
import cPickle as pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.python.platform import gfile
import collections

from run_inference import predict_star, predict
import pandas as pd
import seaborn as sns

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# need to map the 20 nearest neighbors also

## Login and example of using Flickr API

In [12]:
import flickrapi

In [39]:
api_key = u'3fd904198236c855140fb804a819dcab'
api_secret = u'e943b596588106ff'

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')
if not flickr.token_valid(perms=u'write'):

    # Get a request token
    flickr.get_request_token(oauth_callback='oob')

    # Open a browser at the authentication URL. Do this however
    # you want, as long as the user visits that URL.
    authorize_url = flickr.auth_url(perms=u'write')
    print authorize_url 
    
    # Get the verifier code from the user. Do this however you
    # want, as long as the user gives the application the code.
    verifier = unicode(raw_input('Verifier code: '))

    # Trade the request token for an access token
    flickr.get_access_token(verifier)

In [62]:
flickr.photos.addTags(photo_id=11303139385, tags="metal")

'{"tags":{"tag":[{"author":"12403504@N02","raw":"metal","machine_tag":0,"_content":"metal","authorname":"The British Library","full_tag_id":"12383156-11303139385-1042","id":"1042"}]},"stat":"ok"}'

In [43]:
flickr.photos.removeTag(photo_id=11303139385, tag_id="12383156-11303139385-335238")

'{"stat":"ok"}'

In [55]:
print flickr.tags.getListPhoto(photo_id=11303139385)

{"photo":{"id":"11303139385","tags":{"tag":[{"id":"12383156-11303139385-137769045","author":"12403504@N02","authorname":"The British Library","raw":"bldigital","_content":"bldigital","machine_tag":false},{"id":"12383156-11303139385-150790816","author":"12403504@N02","authorname":"The British Library","raw":"date:1895","_content":"date1895","machine_tag":false},{"id":"12383156-11303139385-147784745","author":"12403504@N02","authorname":"The British Library","raw":"pubplace:New York & London","_content":"pubplacenewyorklondon","machine_tag":false},{"id":"12383156-11303139385-136904","author":"12403504@N02","authorname":"The British Library","raw":"public_domain","_content":"publicdomain","machine_tag":false},{"id":"12383156-11303139385-163219536","author":"12403504@N02","authorname":"The British Library","raw":"sysnum:002884871","_content":"sysnum002884871","machine_tag":false},{"id":"12383156-11303139385-162949425","author":"12403504@N02","authorname":"The British Library","raw":"PETERS

## Adding tags

In [10]:
# mapping from index to category
(image_metadata, book_metadata, image_to_idx) = pickle.load(open("/data/all_metadata_1M_tags.pkl", 'r'))

In [11]:
# mapping from index to URL
(idx_to_url, blid_to_url) = pickle.load(open("/data/image_to_url_mappings.pkl"))
len(idx_to_url.keys())

970218

In [ ]:
tag_files = glob.glob("/data/nearest_neighbor_tagging/tags/*_final.pkl")
for fn in tag_files:
    category = os.path.basename(fn)[:-10]
    if category == "people" or category == "animals":
        category = "mammal"
        continue
    print category
    
    img_to_tag = pickle.load(open(fn, 'r'))
    ctr = 0
    for img in img_to_tag:
        if ctr % 2000 == 0: print ctr
        ctr += 1
        flickr_id = int(idx_to_url[img])
        our_tags = ', '.join(["sherlocknet:tag=\"{}\"".format(tag[0]) for tag in img_to_tag[img]])
        our_tags = our_tags + ', sherlocknet:category=\"{}\"'.format(category)
        flickr.photos.addTags(photo_id=flickr_id, tags=our_tags)
    break


diagrams
0


In [30]:
verifier = str(raw_input('Verifier code: '))
print verifier

Verifier code: 2-3
2-3
